In [192]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as scs
import statsmodels.api as sm
import matplotlib.pyplot as plt
import sqlite3
import statsmodels.formula.api as smf

%config InlineBackend.figure_formats = ['svg']
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 100

# Load Data

In [168]:
# Specify categorical columns
df = pd.read_csv('../temperament.csv', na_values=' ',
                 dtype={'re_race': 'category', 'lunch': 'category'})
df = df.set_index('id')
df.head()

,cohort,recruitwave2,w1c_data,w1p_data,t1teacher_id,t1school,t1day1_mo,t1day1_day,t1day1_yr,t1day2_mo,t1day2_day,t1day2_yr,t1age,sex,ethnicity,race,re_race,lunch,birthday,birthda,birthmo,birthyr,t1agemon,t1ageyear,t1day1date,c,t1seq1,t1seq2,t1seq3,t1seq4,t1seq5,t1seq6,t1seq7,t1seq8,t1seq9,t1seq10,t1seq11,t1seq12,t1seq13,t1seq14,t1seq15,t1seq16,t1seq17,t1seq18,t1seq19,t1seq20,t1seq21,mt1victim,mt1overtvic,mt1relationvic,...,t1comprelation,t2compovert,t2comprelation,t1physicalagg,t2physicalagg,re_sex,c_solve,c_t1tovertagg,c_icontrol,solvexic,c_icontrolabove,c_icontrolbelow,solveicabove,solveicbelow,filter_$,c_humor,c_revenge,c_ignore,c_adult,c_negemot,c_t1psmfq,c_t1vic,humorxsex,revengexsex,ignorexsex,solvexsex,adultxsex,humorxnegemot,revengexnegemot,ignorexnegemot,solvexnegemot,adultxnegemot,negemotxsex,negemotxhumorxsex,negemotxrevengexsex,negemotxignorexsex,negemotxsolvexsex,negemotxadultxsex,compovert,c_overt,overtxicontrol,overtxsex,icontrolxsex,overtxicontrolxsex,overtxnegemot,overtxnegemotxsex,Zmt1overtvic,Zmt1tovertvic,ZSco01,ZSco02
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6085,1,0,1,1,13.0,6.0,3.0,2.0,2006.0,3.0,6.0,2006.0,8.208077,0,2,2,1,1,12/16/1997,16,12,1997,2.0,8.0,3/2/2006,1,4.0,2.0,3.0,4.0,3.0,1.0,2.0,1.0,1.0,3.0,1.0,1.0,2.0,1.0,3.0,5.0,4.0,2.0,1.0,1.0,3.0,2.285714,2.818182,1.7,...,NaN,2.610303,-0.769878,3.333333,3.666667,-1,-0.049270,1.9726,0.0375,0.003278,-0.61316,0.51796,0.042229,-0.035672,1,NaN,0.045067,-0.064433,0.004178,-0.099333,-0.187377,0.078314,NaN,-0.054667,0.064433,0.061470,-0.004178,NaN,-0.009772,0.006800,0.010683,-0.000441,0.0,NaN,0.005769,-0.006800,-0.006487,0.000441,1.580884,1.580884,0.059136,0.0,0.0,0.0,-0.157034,0.0,0.646884,0.930064,0.641788,0.939096
11005,2,0,1,1,61.0,11.0,2.0,19.0,2007.0,2.0,27.0,2007.0,7.824778,0,2,1,0,1,4/24/1999,24,4,1999,9.0,7.0,2/19/2007,1,2.0,1.0,2.0,3.0,3.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,3.0,2.0,2.0,2.0,1.0,1.0,1.761905,1.909091,1.6,...,NaN,NaN,NaN,1.000000,1.000000,-1,0.116485,-0.2774,-0.4625,-0.053054,-1.11316,0.01796,-0.107848,0.001740,0,-0.033515,-0.054985,0.071977,-0.016677,0.286778,0.120315,-0.445495,0.033515,0.045385,-0.071977,-0.104285,0.016677,-0.009404,-0.009308,0.020195,0.019718,-0.004679,0.0,0.009404,0.012734,-0.020195,-0.029260,0.004679,NaN,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN
11008,2,0,1,1,61.0,11.0,2.0,5.0,2007.0,2.0,19.0,2007.0,7.427789,0,2,2,1,0,9/2/1999,2,9,1999,5.0,7.0,2/5/2007,1,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.285714,1.181818,1.4,...,NaN,NaN,NaN,1.000000,1.000000,-1,0.132918,-0.2774,0.5375,0.051265,-0.11316,1.01796,-0.012823,0.115353,0,-0.039809,-0.062327,0.062536,-0.002691,-0.179889,-0.264300,-0.921686,0.039809,0.052727,-0.062536,-0.120718,0.002691,0.007408,0.018535,-0.011637,-0.039528,0.000501,0.0,-0.007408,-0.009812,0.011637,0.022464,-0.000501,NaN,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN
11009,2,0,1,1,61.0,11.0,2.0,5.0,2007.0,2.0,19.0,2007.0,8.295688,0,2,2,1,1,10/20/1998,20,10,1998,3.0,8.0,2/5/2007,1,3.0,3.0,3.0,1.0,2.0,5.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,3.0,4.0,3.0,3.0,1.0,1.0,1.0,2.047619,2.090909,2.0,...,NaN,NaN,NaN,1.000000,4.666667,-1,0.004621,-0.2774,-0.3375,0.006330,-0.98816,0.14296,0.014802,-0.002141,0,-0.064393,0.120259,-0.030677,-0.009093,0.631222,-0.033531,-0.159781,0.064393,-0.129859,0.030677,0.007579,0.009093,-0.040247,0.061780,-0.019174,0.002374,-0.005683,0.0,0.040247,-0.081165,0.019174,0.004737,0.005683,NaN,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN
11011,2,0,1,1,61.0,11.0,2.0,5.0,2007.0,2.0,19.0,2007.0,7.696099,0,2,2,1,0,5/27/1999,27,5,1999,8.0,7.0,2/5/2007,1,2.0,3.0,4.0,2.0,4.0,3.0,1.0,1.0,4.0,3.0,4.0,5.0,1.0,3.0,2.0,2.0,4.0,2.0,3.0,4.0,1.0,2.761905,2.727273,2.8,...,NaN,NaN,NaN,1.000000,3.000000,-1,0.009721,-0.2774,-0.7125,0.007880,-1.36316,-0.23204,0.013467,0.002292,0,-0.045452,0.034538,-0.049376,0.078814,0.620111,0.043392,0.554505,0.045452,-0.044138,0.049376,0.002479,-0.078814,-0.027903,0.01

# Select rows for Cohort 1 with no missing data

## For analyses on aggression

In [170]:
aggdf = df[(df.cohort == 1) & (~df.mt1tovertagg.isnull()) & (~df.mt2tovertagg.isnull()) &
           (~df.mt1overtvic.isnull()) & (~df.mt1tovertvic.isnull()) & (~df.mt1picontrol.isnull())]
aggdf.head()

,cohort,recruitwave2,w1c_data,w1p_data,t1teacher_id,t1school,t1day1_mo,t1day1_day,t1day1_yr,t1day2_mo,t1day2_day,t1day2_yr,t1age,sex,ethnicity,race,re_race,lunch,birthday,birthda,birthmo,birthyr,t1agemon,t1ageyear,t1day1date,c,t1seq1,t1seq2,t1seq3,t1seq4,t1seq5,t1seq6,t1seq7,t1seq8,t1seq9,t1seq10,t1seq11,t1seq12,t1seq13,t1seq14,t1seq15,t1seq16,t1seq17,t1seq18,t1seq19,t1seq20,t1seq21,mt1victim,mt1overtvic,mt1relationvic,...,t1comprelation,t2compovert,t2comprelation,t1physicalagg,t2physicalagg,re_sex,c_solve,c_t1tovertagg,c_icontrol,solvexic,c_icontrolabove,c_icontrolbelow,solveicabove,solveicbelow,filter_$,c_humor,c_revenge,c_ignore,c_adult,c_negemot,c_t1psmfq,c_t1vic,humorxsex,revengexsex,ignorexsex,solvexsex,adultxsex,humorxnegemot,revengexnegemot,ignorexnegemot,solvexnegemot,adultxnegemot,negemotxsex,negemotxhumorxsex,negemotxrevengexsex,negemotxignorexsex,negemotxsolvexsex,negemotxadultxsex,compovert,c_overt,overtxicontrol,overtxsex,icontrolxsex,overtxicontrolxsex,overtxnegemot,overtxnegemotxsex,Zmt1overtvic,Zmt1tovertvic,ZSco01,ZSco02
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6085,1,0,1,1,13.0,6.0,3.0,2.0,2006.0,3.0,6.0,2006.0,8.208077,0,2,2,1,1,12/16/1997,16,12,1997,2.0,8.0,3/2/2006,1,4.0,2.0,3.0,4.0,3.0,1.0,2.0,1.0,1.0,3.0,1.0,1.0,2.0,1.0,3.0,5.0,4.0,2.0,1.0,1.0,3.0,2.285714,2.818182,1.7,...,NaN,2.610303,-0.769878,3.333333,3.666667,-1,-0.049270,1.9726,0.0375,0.003278,-0.61316,0.51796,0.042229,-0.035672,1,NaN,0.045067,-0.064433,0.004178,-0.099333,-0.187377,0.078314,NaN,-0.054667,0.064433,0.061470,-0.004178,NaN,-0.009772,0.006800,0.010683,-0.000441,0.0,NaN,0.005769,-0.006800,-0.006487,0.000441,1.580884,1.580884,0.059136,0.0,0.0,0.0,-0.157034,0.0,0.646884,0.930064,0.641788,0.939096
1002,1,0,1,1,17.0,1.0,1.0,11.0,2006.0,1.0,11.0,2006.0,7.039014,0,2,2,1,0,12/28/1998,28,12,1998,0.0,7.0,1/11/2006,1,5.0,1.0,5.0,1.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,5.0,3.0,5.0,5.0,5.0,5.0,3.571429,4.818182,2.2,...,-1.135892,1.852989,0.635096,1.000000,1.000000,-1,-0.088186,-0.2774,-0.0875,0.018604,-0.73816,0.39296,0.079563,-0.042355,1,-0.041757,-0.082457,-0.011814,-0.040029,-0.921556,-0.264300,1.364029,0.041757,0.072857,0.011814,0.100386,0.040029,0.038740,0.085678,0.010961,0.091630,0.037137,0.0,-0.038740,-0.067594,-0.010961,-0.093133,-0.037137,2.498208,2.498208,-0.220192,0.0,0.0,0.0,-2.302237,0.0,2.909522,-0.393039,2.883833,-0.385625
1004,1,0,1,1,17.0,1.0,1.0,11.0,2006.0,1.0,11.0,2006.0,7.468857,0,2,6,0,0,7/24/1998,24,7,1998,5.0,7.0,1/11/2006,1,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,1.0,1.0,3.0,3.0,1.476190,1.909091,1.0,...,-1.619677,0.999122,0.910435,1.000000,2.666667,-1,0.089335,-0.2774,0.0375,-0.003319,-0.61316,0.51796,-0.042759,0.036120,1,-0.016547,-0.086659,-0.012865,0.009341,-0.274333,-0.264300,-0.731210,0.016547,0.077059,0.012865,-0.077135,-0.009341,0.004642,0.033956,0.003609,-0.035005,-0.002621,0.0,-0.004642,-0.021618,-0.003609,0.021639,0.002621,-0.762949,-0.762949,-0.029049,0.0,0.0,0.0,0.209302,0.0,-0.381589,-0.393039,-0.377324,-0.385625
1006,1,0,1,1,17.0,1.0,1.0,11.0,2006.0,1.0,11.0,2006.0,7.690623,0,2,2,1,1,5/4/1998,4,5,1998,8.0,7.0,1/11/2006,1,5.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,5.0,4.0,5.0,4.523810,4.727273,4.3,...,2.452864,-0.439749,-0.332146,1.000000,1.000000,-1,0.021370,-0.2774,-0.4625,-0.000969,-1.11316,0.01796,-0.001971,0.000032,1,-0.067819,0.067157,-0.060559,0.041535,0.622889,0.120315,2.316410,0.067819,-0.076757,0.060559,-0.009170,-0.041535,-0.041823,0.033940,-0.037346,0.010800,0.025614,0.0,0.041823,-0.047335,0.037346,-0.005655,-0.025614,1.912953,1.912953,-0.893033,0.0,0.0,0.0,1.191557,0.0,2.806675,-0.875793,2.781922,-0.868969
1007,1,0,1,1,17.0,1.0,1.0,9.0,2006.0,1.0,11.0,2006.0,7.403149,0,2,3,0,0,8/15/1998,15,8,1998,4.0,7.0,1/9/2006,1,3.0,1.0,5.0,1.0,2.0,3.0,1.0,2.0,1.0,1.0,2.0,1.0,4.0,1.0,2.0,1.0,2.0,1.0,4.0,1.0,2.0,1.952381,2.090909,1.8,...,0.739859,0.489876,-0.021501,1.666667,2.000000,-1,-

## For analyses on depression

In [171]:
depdf = df[(df.cohort == 1) & (~df.mt1psmfq.isnull()) & (~df.mt2psmfq.isnull()) &
           (~df.mt1overtvic.isnull()) & (~df.mt1tovertvic.isnull()) & (~df.mt1pnegemot.isnull())]
depdf.head()

,cohort,recruitwave2,w1c_data,w1p_data,t1teacher_id,t1school,t1day1_mo,t1day1_day,t1day1_yr,t1day2_mo,t1day2_day,t1day2_yr,t1age,sex,ethnicity,race,re_race,lunch,birthday,birthda,birthmo,birthyr,t1agemon,t1ageyear,t1day1date,c,t1seq1,t1seq2,t1seq3,t1seq4,t1seq5,t1seq6,t1seq7,t1seq8,t1seq9,t1seq10,t1seq11,t1seq12,t1seq13,t1seq14,t1seq15,t1seq16,t1seq17,t1seq18,t1seq19,t1seq20,t1seq21,mt1victim,mt1overtvic,mt1relationvic,...,t1comprelation,t2compovert,t2comprelation,t1physicalagg,t2physicalagg,re_sex,c_solve,c_t1tovertagg,c_icontrol,solvexic,c_icontrolabove,c_icontrolbelow,solveicabove,solveicbelow,filter_$,c_humor,c_revenge,c_ignore,c_adult,c_negemot,c_t1psmfq,c_t1vic,humorxsex,revengexsex,ignorexsex,solvexsex,adultxsex,humorxnegemot,revengexnegemot,ignorexnegemot,solvexnegemot,adultxnegemot,negemotxsex,negemotxhumorxsex,negemotxrevengexsex,negemotxignorexsex,negemotxsolvexsex,negemotxadultxsex,compovert,c_overt,overtxicontrol,overtxsex,icontrolxsex,overtxicontrolxsex,overtxnegemot,overtxnegemotxsex,Zmt1overtvic,Zmt1tovertvic,ZSco01,ZSco02
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
6085,1,0,1,1,13.0,6.0,3.0,2.0,2006.0,3.0,6.0,2006.0,8.208077,0,2,2,1,1,12/16/1997,16,12,1997,2.0,8.0,3/2/2006,1,4.0,2.0,3.0,4.0,3.0,1.0,2.0,1.0,1.0,3.0,1.0,1.0,2.0,1.0,3.0,5.0,4.0,2.0,1.0,1.0,3.0,2.285714,2.818182,1.7,...,NaN,2.610303,-0.769878,3.333333,3.666667,-1,-0.049270,1.9726,0.0375,0.003278,-0.61316,0.51796,0.042229,-0.035672,1,NaN,0.045067,-0.064433,0.004178,-0.099333,-0.187377,0.078314,NaN,-0.054667,0.064433,0.061470,-0.004178,NaN,-0.009772,0.006800,0.010683,-0.000441,0.0,NaN,0.005769,-0.006800,-0.006487,0.000441,1.580884,1.580884,0.059136,0.0,0.0,0.0,-0.157034,0.0,0.646884,0.930064,0.641788,0.939096
6019,1,0,1,1,9.0,6.0,2.0,9.0,2006.0,2.0,23.0,2006.0,8.599589,0,2,2,1,0,7/5/1997,5,7,1997,7.0,8.0,2/9/2006,1,2.0,3.0,2.0,2.0,3.0,2.0,1.0,3.0,3.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.952381,1.727273,2.2,...,NaN,NaN,NaN,1.000000,NaN,-1,0.060936,-0.2774,0.1625,0.003199,-0.48816,0.64296,-0.020179,0.026577,1,0.042903,-0.071567,0.046441,-0.064092,-0.038222,-0.110454,-0.255019,-0.042903,0.061967,-0.046441,-0.048736,0.064092,-0.001906,0.011145,-0.002063,-0.009489,0.002847,0.0,0.001906,-0.002753,0.002063,0.002165,-0.002847,-0.322312,-0.322312,NaN,0.0,0.0,0.0,0.012319,0.0,NaN,NaN,-0.581146,0.258834
1002,1,0,1,1,17.0,1.0,1.0,11.0,2006.0,1.0,11.0,2006.0,7.039014,0,2,2,1,0,12/28/1998,28,12,1998,0.0,7.0,1/11/2006,1,5.0,1.0,5.0,1.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,1.0,1.0,1.0,1.0,5.0,3.0,5.0,5.0,5.0,5.0,3.571429,4.818182,2.2,...,-1.135892,1.852989,0.635096,1.000000,1.000000,-1,-0.088186,-0.2774,-0.0875,0.018604,-0.73816,0.39296,0.079563,-0.042355,1,-0.041757,-0.082457,-0.011814,-0.040029,-0.921556,-0.264300,1.364029,0.041757,0.072857,0.011814,0.100386,0.040029,0.038740,0.085678,0.010961,0.091630,0.037137,0.0,-0.038740,-0.067594,-0.010961,-0.093133,-0.037137,2.498208,2.498208,-0.220192,0.0,0.0,0.0,-2.302237,0.0,2.909522,-0.393039,2.883833,-0.385625
1004,1,0,1,1,17.0,1.0,1.0,11.0,2006.0,1.0,11.0,2006.0,7.468857,0,2,6,0,0,7/24/1998,24,7,1998,5.0,7.0,1/11/2006,1,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,3.0,1.0,1.0,3.0,3.0,1.476190,1.909091,1.0,...,-1.619677,0.999122,0.910435,1.000000,2.666667,-1,0.089335,-0.2774,0.0375,-0.003319,-0.61316,0.51796,-0.042759,0.036120,1,-0.016547,-0.086659,-0.012865,0.009341,-0.274333,-0.264300,-0.731210,0.016547,0.077059,0.012865,-0.077135,-0.009341,0.004642,0.033956,0.003609,-0.035005,-0.002621,0.0,-0.004642,-0.021618,-0.003609,0.021639,0.002621,-0.762949,-0.762949,-0.029049,0.0,0.0,0.0,0.209302,0.0,-0.381589,-0.393039,-0.377324,-0.385625
1006,1,0,1,1,17.0,1.0,1.0,11.0,2006.0,1.0,11.0,2006.0,7.690623,0,2,2,1,1,5/4/1998,4,5,1998,8.0,7.0,1/11/2006,1,5.0,3.0,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,5.0,4.0,5.0,4.523810,4.727273,4.3,...,2.452864,-0.439749,-0.332146,1.000000,1.000000,-1,0.021370,-0.2774,-0.4625,-0.000969,-1.1

# Select necessary columns for the analyses

## For analyses on aggression

In [172]:
oaggdf = aggdf[['t1age', 're_race', 'lunch', 'sex', 'mt1tovertagg', 'mt2tovertagg',
                'mt1overtvic', 'mt1tovertvic', 'mt1picontrol']]
oaggdf = oaggdf.copy()

In [173]:
oaggdf.head()

,t1age,re_race,lunch,sex,mt1tovertagg,mt2tovertagg,mt1overtvic,mt1tovertvic,mt1picontrol
id,,,,,,,,,
6085,8.208077,1,1,0,3.25,3.75,2.818182,2.111111,3.375
1002,7.039014,1,0,0,1.00,1.00,4.818182,1.363636,3.250
1004,7.468857,0,0,0,1.00,2.50,1.909091,1.363636,3.375
1006,7.690623,1,1,0,1.00,1.00,4.727273,1.090909,2.875
1007,7.403149,0,0,0,1.50,2.00,2.090909,1.636364,3.125


In [174]:
oaggdf.dtypes

t1age            float64
re_race         category
lunch           category
sex                int64
mt1tovertagg     float64
mt2tovertagg     float64
mt1overtvic      float64
mt1tovertvic     float64
mt1picontrol     float64
dtype: object

In [175]:
oaggdf.shape

(283, 9)

In [176]:
oaggdf.describe()

,t1age,sex,mt1tovertagg,mt2tovertagg,mt1overtvic,mt1tovertvic,mt1picontrol
count,283.000000,283.000000,283.000000,283.000000,283.000000,283.000000,283.000000
mean,7.945891,0.558304,1.448763,1.552120,2.246386,1.585680,3.337498
std,0.319879,0.497469,0.893747,0.946098,0.883924,0.564941,0.585106
min,7.039014,0.000000,1.000000,1.000000,1.000000,1.000000,1.375000
25%,7.691992,0.000000,1.000000,1.000000,1.545455,1.181818,3.000000
50%,7.967146,1.000000,1.000000,1.000000,2.090909,1.454545,3.375000
75%,8.171116,1.000000,1.500000,1.750000,2.636364,1.818182,3.750000
max,9.117043,1.000000,5.000000,5.000000,4.818182,4.181818,4.875000


## For analyses on depression

In [177]:
depdf = depdf[['t1age', 're_race', 'lunch', 'sex', 'mt1psmfq', 'mt2psmfq',
                'mt1overtvic', 'mt1tovertvic', 'mt1pnegemot']]
depdf.head()

,t1age,re_race,lunch,sex,mt1psmfq,mt2psmfq,mt1overtvic,mt1tovertvic,mt1pnegemot
id,,,,,,,,,
6085,8.208077,1,1,0,1.076923,1.153846,2.818182,2.111111,2.516667
6019,8.599589,1,0,0,1.153846,1.230769,1.727273,1.727273,2.577778
1002,7.039014,1,0,0,1.000000,1.000000,4.818182,1.363636,1.694444
1004,7.468857,0,0,0,1.000000,1.000000,1.909091,1.363636,2.341667
1006,7.690623,1,1,0,1.384615,1.538462,4.727273,1.090909,3.238889


In [178]:
depdf.dtypes

t1age            float64
re_race         category
lunch           category
sex                int64
mt1psmfq         float64
mt2psmfq         float64
mt1overtvic      float64
mt1tovertvic     float64
mt1pnegemot      float64
dtype: object

In [179]:
depdf.shape

(242, 9)

In [180]:
depdf.mt2psmfq.count()

242

In [181]:
depdf.describe()

,t1age,sex,mt1psmfq,mt2psmfq,mt1overtvic,mt1tovertvic,mt1pnegemot
count,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000
mean,7.937912,0.561983,1.224650,1.224968,2.245680,1.581225,2.616007
std,0.326755,0.497171,0.292352,0.263350,0.892043,0.564251,0.606103
min,7.039014,0.000000,1.000000,1.000000,1.000000,1.000000,1.100000
25%,7.682409,0.000000,1.000000,1.000000,1.545455,1.090909,2.187500
50%,7.964408,1.000000,1.153846,1.153846,2.090909,1.454545,2.545833
75%,8.168378,1.000000,1.307692,1.307692,2.636364,1.818182,3.013194
max,9.117043,1.000000,2.923077,2.750000,4.818182,4.181818,4.344444


# Creating composite scores

In [182]:
oaggdf['std_mt1overtvic'] = (oaggdf['mt1overtvic'] - oaggdf['mt1overtvic'].mean()) / oaggdf['mt1overtvic'].std()
oaggdf.head()

,t1age,re_race,lunch,sex,mt1tovertagg,mt2tovertagg,mt1overtvic,mt1tovertvic,mt1picontrol,std_mt1overtvic
id,,,,,,,,,,
6085,8.208077,1,1,0,3.25,3.75,2.818182,2.111111,3.375,0.646884
1002,7.039014,1,0,0,1.00,1.00,4.818182,1.363636,3.250,2.909522
1004,7.468857,0,0,0,1.00,2.50,1.909091,1.363636,3.375,-0.381589
1006,7.690623,1,1,0,1.00,1.00,4.727273,1.090909,2.875,2.806675
1007,7.403149,0,0,0,1.50,2.00,2.090909,1.636364,3.125,-0.175894


In [183]:
oaggdf.std_mt1overtvic.describe()

count    2.830000e+02
mean    -3.138440e-17
std      1.000000e+00
min     -1.410061e+00
25%     -7.929774e-01
50%     -1.758942e-01
75%      4.411891e-01
max      2.909522e+00
Name: std_mt1overtvic, dtype: float64

In [184]:
oaggdf['std_mt1tovertvic'] = (oaggdf['mt1tovertvic'] - oaggdf['mt1tovertvic'].mean()) / oaggdf['mt1tovertvic'].std()

In [185]:
oaggdf.head()

,t1age,re_race,lunch,sex,mt1tovertagg,mt2tovertagg,mt1overtvic,mt1tovertvic,mt1picontrol,std_mt1overtvic,std_mt1tovertvic
id,,,,,,,,,,,
6085,8.208077,1,1,0,3.25,3.75,2.818182,2.111111,3.375,0.646884,0.930064
1002,7.039014,1,0,0,1.00,1.00,4.818182,1.363636,3.250,2.909522,-0.393039
1004,7.468857,0,0,0,1.00,2.50,1.909091,1.363636,3.375,-0.381589,-0.393039
1006,7.690623,1,1,0,1.00,1.00,4.727273,1.090909,2.875,2.806675,-0.875793
1007,7.403149,0,0,0,1.50,2.00,2.090909,1.636364,3.125,-0.175894,0.089715


In [186]:
oaggdf.std_mt1tovertvic.describe()

count    2.830000e+02
mean     3.263977e-16
std      1.000000e+00
min     -1.036711e+00
25%     -7.148750e-01
50%     -2.321211e-01
75%      4.115506e-01
max      4.595417e+00
Name: std_mt1tovertvic, dtype: float64

In [187]:
oaggdf['comp_overt'] = oaggdf['std_mt1overtvic'] + oaggdf['std_mt1tovertvic']

In [188]:
oaggdf['c_overt'] = oaggdf['comp_overt'] - oaggdf['comp_overt'].mean()
oaggdf['c_icontrol'] = oaggdf['mt1picontrol'] - oaggdf['mt1picontrol'].mean()

In [189]:
oaggdf.describe()

,t1age,sex,mt1tovertagg,mt2tovertagg,mt1overtvic,mt1tovertvic,mt1picontrol,std_mt1overtvic,std_mt1tovertvic,comp_overt,c_overt,c_icontrol
count,283.000000,283.000000,283.000000,283.000000,283.000000,283.000000,283.000000,2.830000e+02,2.830000e+02,2.830000e+02,283.000000,2.830000e+02
mean,7.945891,0.558304,1.448763,1.552120,2.246386,1.585680,3.337498,-3.138440e-17,3.263977e-16,3.012902e-16,0.000000,-3.138440e-17
std,0.319879,0.497469,0.893747,0.946098,0.883924,0.564941,0.585106,1.000000e+00,1.000000e+00,1.551871e+00,1.551871,5.851056e-01
min,7.039014,0.000000,1.000000,1.000000,1.000000,1.000000,1.375000,-1.410061e+00,-1.036711e+00,-2.446772e+00,-2.446772,-1.962498e+00
25%,7.691992,0.000000,1.000000,1.000000,1.545455,1.181818,3.000000,-7.929774e-01,-7.148750e-01,-1.127946e+00,-1.127946,-3.374979e-01
50%,7.967146,1.000000,1.000000,1.000000,2.090909,1.454545,3.375000,-1.758942e-01,-2.321211e-01,-4.080153e-01,-0.408015,3.750210e-02
75%,8.171116,1.000000,1.500000,1.750000,2.636364,1.818182,3.750000,4.411891e-01,4.115506e-01,8.188107e-01,0.818811,4.125021e-01
max,9.117043,1.000000,5.000000,5.000000,4.818182,4.181818,4.875000,2.909522e+00,4.595417e+00,5.036606e+00,5.036606,1.537502e+00


# Creating interaction terms

In [190]:
oaggdf['overtxicontrol'] = oaggdf['c_overt'] * oaggdf['c_icontrol']

oaggdf['overtxsex'] = oaggdf['c_overt'] * oaggdf['sex']

oaggdf['icontrolsex'] = oaggdf['c_icontrol'] * oaggdf['sex']

oaggdf['overtxicontrolxsex'] = oaggdf['c_overt'] * oaggdf['c_icontrol'] * oaggdf['sex']


# Regression

## Overt victimization predicting overt aggression

In [195]:
oaggovic = smf.ols(formula=f'mt2tovertagg ~ mt1tovertagg + c_overt + c_icontrol + sex + overtxicontrol + \
                           overtxsex + icontrolsex + overtxicontrolxsex', data=oaggdf).fit()
oaggovic.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           mt2tovertagg   R-squared:                       0.409
Model:                            OLS   Adj. R-squared:                  0.392
Method:                 Least Squares   F-statistic:                     23.69
Date:                Thu, 28 Mar 2019   Prob (F-statistic):           1.66e-27
Time:                        21:00:01   Log-Likelihood:                -310.99
No. Observations:                 283   AIC:                             640.0
Df Residuals:                     274   BIC:                             672.8
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              0.9095      0.117      7.770      0.000       0.679       1.140
mt1tovertagg           0.5282      0.060      8.803      0.000       0.410       0.646
c_overt               -0.0214      0.056     -0.381      0.703      -0.132       0.089
c_icontrol            -0.2670      0.115     -2.313      0.021      -0.494      -0.040
sex                   -0.2179      0.093     -2.334      0.020      -0.402      -0.034
overtxicontrol         0.0907      0.086      1.055      0.293      -0.079       0.260
overtxsex              0.1252      0.066      1.904      0.058      -0.004       0.255
icontrolsex           -0.0346      0.158     -0.218      0.827      -0.346       0.277
overtxicontrolxsex    -0.2432      0.104     -2.334      0.020      -0.448      -0.038
==============================================================================
Omnibus:                       58.083   Durbin-Watson:                   2.050
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              104.946
Skew:                           1.101   Prob(JB):                     1.63e-23
Kurtosis:                       5.012   Cond. No.                         8.82
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Decomposing the 3-way interaction by sex

In [196]:
boys = oaggdf[(oaggdf.sex == 0)]
boys.head()

,t1age,re_race,lunch,sex,mt1tovertagg,mt2tovertagg,mt1overtvic,mt1tovertvic,mt1picontrol,std_mt1overtvic,std_mt1tovertvic,comp_overt,c_overt,c_icontrol,overtxicontrol,overtxsex,icontrolsex,overtxicontrolxsex
id,,,,,,,,,,,,,,,,,,
6085,8.208077,1,1,0,3.25,3.75,2.818182,2.111111,3.375,0.646884,0.930064,1.576948,1.576948,0.037502,0.059139,0.0,0.0,0.0
1002,7.039014,1,0,0,1.00,1.00,4.818182,1.363636,3.250,2.909522,-0.393039,2.516483,2.516483,-0.087498,-0.220187,0.0,-0.0,-0.0
1004,7.468857,0,0,0,1.00,2.50,1.909091,1.363636,3.375,-0.381589,-0.393039,-0.774628,-0.774628,0.037502,-0.029050,-0.0,0.0,-0.0
1006,7.690623,1,1,0,1.00,1.00,4.727273,1.090909,2.875,2.806675,-0.875793,1.930882,1.930882,-0.462498,-0.893029,0.0,-0.0,-0.0
1007,7.403149,0,0,0,1.50,2.00,2.090909,1.636364,3.125,-0.175894,0.089715,-0.086179,-0.086179,-0.212498,0.018313,-0.0,-0.0,0.0


In [197]:
boys.shape

(125, 18)

In [198]:
boys['std_mt1overtvic'] = (boys['mt1overtvic'] - boys['mt1overtvic'].mean()) / boys['mt1overtvic'].std()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [199]:
boys['std_mt1tovertvic'] = (boys['mt1tovertvic'] - boys['mt1tovertvic'].mean()) / boys['mt1tovertvic'].std()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [200]:
boys['comp_overt'] = boys['std_mt1overtvic'] + boys['std_mt1tovertvic']

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [201]:
boys['c_overt'] = boys['comp_overt'] - boys['comp_overt'].mean()
boys['c_icontrol'] = boys['mt1picontrol'] - boys['mt1picontrol'].mean()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [202]:
boys.describe()

,t1age,sex,mt1tovertagg,mt2tovertagg,mt1overtvic,mt1tovertvic,mt1picontrol,std_mt1overtvic,std_mt1tovertvic,comp_overt,c_overt,c_icontrol,overtxicontrol,overtxsex,icontrolsex,overtxicontrolxsex
count,125.000000,125.0,125.000000,125.000000,125.000000,125.000000,125.000000,1.250000e+02,1.250000e+02,1.250000e+02,1.250000e+02,1.250000e+02,125.000000,125.0,125.0,125.0
mean,7.952712,0.0,1.672000,1.800000,2.230545,1.707071,3.220095,-3.894662e-16,-1.136868e-16,-4.796163e-16,-2.486900e-17,5.471179e-16,-0.218361,0.0,0.0,0.0
std,0.365467,0.0,0.994144,1.041478,0.846570,0.616665,0.590608,1.000000e+00,1.000000e+00,1.471588e+00,1.471588e+00,5.906077e-01,0.882835,0.0,0.0,0.0
min,7.039014,0.0,1.000000,1.000000,1.000000,1.000000,1.375000,-1.453565e+00,-1.146605e+00,-2.492785e+00,-2.492785e+00,-1.845095e+00,-4.505825,0.0,0.0,0.0
25%,7.649555,0.0,1.000000,1.000000,1.636364,1.272727,2.875000,-7.018694e-01,-7.043428e-01,-1.031300e+00,-1.031300e+00,-3.450952e-01,-0.483298,0.0,-0.0,-0.0
50%,7.942505,0.0,1.000000,1.250000,2.090909,1.636364,3.250000,-1.649436e-01,-1.146605e-01,-4.015824e-01,-4.015824e-01,2.990476e-02,-0.080856,0.0,0.0,0.0
75%,8.216290,0.0,2.250000,2.500000,2.545455,2.090909,3.625000,3.719822e-01,6.224424e-01,9.562612e-01,9.562612e-01,4.049048e-01,0.099042,0.0,0.0,0.0
max,9.117043,0.0,5.000000,5.000000,4.818182,4.181818,4.750000,3.056611e+00,4.013116e+00,4.492483e+00,4.492483e+00,1.529905e+00,2.397487,0.0,0.0,0.0


In [203]:
boys['overtxicontrol'] = boys['c_overt'] * boys['c_icontrol']


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [205]:
b_oaggovic = smf.ols(formula=f'mt2tovertagg ~ mt1tovertagg + c_overt + c_icontrol + overtxicontrol', data=boys).fit()
b_oaggovic.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           mt2tovertagg   R-squared:                       0.308
Model:                            OLS   Adj. R-squared:                  0.285
Method:                 Least Squares   F-statistic:                     13.36
Date:                Thu, 28 Mar 2019   Prob (F-statistic):           4.90e-09
Time:                        21:35:45   Log-Likelihood:                -158.92
No. Observations:                 125   AIC:                             327.8
Df Residuals:                     120   BIC:                             342.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.8467      0.185      4.587      0.000       0.481       1.212
mt1tovertagg       0.5807      0.099      5.854      0.000       0.384       0.777
c_overt           -0.0527      0.069     -0.763      0.447      -0.190       0.084
c_icontrol        -0.2411      0.139     -1.735      0.085      -0.516       0.034
overtxicontrol     0.0934      0.103      0.910      0.365      -0.110       0.297
==============================================================================
Omnibus:                       18.180   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               21.325
Skew:                           0.987   Prob(JB):                     2.34e-05
Kurtosis:                       3.442   Cond. No.                         5.89
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""